In [1]:
import numpy as np
import sys
import json
import time
import datetime
from epics import caget, caput, PV

from beam_io import *
from emittance_calc import get_normemit

import matplotlib.pyplot as plt

In [2]:
caget("OTRS:IN20:571:RESOLUTION") #12.23

12.23

In [7]:
PV("OTRS:IN20:571:IMAGE").get()

array([10,  2,  5, ..., 13,  8, 11], dtype=int32)

In [3]:
# testing for bax
get_beamsize_inj(use_profMon=True)

array([3.55954846e-05, 2.60920621e-05])

In [5]:
pv_info = json.load(open('pv_info.json'))

im_pv = PV("OTRS:IN20:571:IMAGE")
n_col_pv =  PV("OTRS:IN20:571:ROI_YNP")
n_row_pv =  PV("OTRS:IN20:571:ROI_XNP")

meas_input_pv =  PV(pv_info['device']['QUAD']['Q525'])
varx_pv =  PV(pv_info['device']['SOL']['SOL121'])
vary_pv =  PV(pv_info['device']['QUAD']['Q121'])
varz_pv =  PV(pv_info['device']['QUAD']['Q122'])

varx_act_pv =  PV("SOLN:IN20:121:BACT")
vary_act_pv =  PV("QUAD:IN20:121:BACT")
varz_act_pv =  PV("QUAD:IN20:122:BACT")
quad_act_pv =  PV("QUAD:IN20:525:BACT")

x_size_pv = PV(pv_info['device']['OTR2']['profmonxsize'])
y_size_pv = PV(pv_info['device']['OTR2']['profmonysize'])

energy = caget(pv_info['energy']['DL1'])
resolution = 12.23*1e-6

In [3]:
pvname_list = [PV('SOLN:IN20:121:BCTRL'), #solenoid
               PV('QUAD:IN20:121:BCTRL'), #skew quad
               PV('QUAD:IN20:122:BCTRL'), #skew quad
               PV('QUAD:IN20:525:BCTRL')  #matching quad
              ]

In [4]:
# init vals
init_sol  = pvname_list[0].get()
init_q121 = pvname_list[1].get()
init_q122 = pvname_list[2].get()
init_q525 = pvname_list[3].get()
print(init_sol, init_q121, init_q122, init_q525)

0.4704728 0.0076648469 0.0034501273 -2.9159415


In [8]:
def set_injector(varx, vary, varz):
    pvname_list[0].put(varx)
    pvname_list[1].put(vary)
    pvname_list[2].put(vary)

## Using images

In [ ]:
# example init quad list
quad_list = [-5, -4, -3, -2]
xrms = []
yrms = []
xrms_err = []
yrms_err = []

for quad in quad_list:
    setquad(quad)
    time.sleep(3)
    beamsize = get_updated_beamsizes(quad, use_profMon=False) 
    xrms.append(beamsize[0])
    yrms.append(beamsize[1])
    xrms_err.append(beamsize[2])
    yrms_err.append(beamsize[3])
    
# # Compute and return emittance using quad_list and beamsizes_list
emittance = get_normemit(0.135, quad, quad, xrms, yrms, xrms_err, yrms_err, \
                           adapt_ranges=True, num_points=5, show_plots=True)

## Using profile Monitor

In [ ]:
# example init quad list
quad_list = [-5, -4, -3, -2]
xrms = []
yrms = []
xrms_err = []
yrms_err = []

for quad in quad_list:
    setquad(quad)
    time.sleep(3)
    beamsize = get_updated_beamsizes(quad, use_profMon=True) 
    xrms.append(beamsize[0])
    yrms.append(beamsize[1])
    xrms_err.append(beamsize[2])
    yrms_err.append(beamsize[3])
    
# # Compute and return emittance using quad_list and beamsizes_list
emittance = get_normemit(0.135, quad, quad, xrms, yrms, xrms_err, yrms_err, \
                           adapt_ranges=True, num_points=5, show_plots=True)